<a href="https://colab.research.google.com/github/zeynepmerve97/FaceRecognitionWithWebcam/blob/main/FaceRecognitionwithWebcamipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Face Recognition with Webcam

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import pandas as pd
import cv2

First Model

In [ ]:
model_first = Sequential()
model_first.add(Conv2D(32, kernel_size = (5,5),padding = 'Same',strides=(2, 2),
                 activation ='relu', input_shape = (50,50,3) ))
model_first.add(Conv2D(64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model_first.add(MaxPooling2D(5,5,padding='same'))
model_first.add(Dropout(0.25))

model_first.add(Conv2D(128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model_first.add(Conv2D(256, kernel_size = (5,5),padding = 'Same', 
                  
                 activation ='relu'))

model_first.add(Dropout(0.50))

model_first.add(Flatten())
model_first.add(Dense(128, activation = "relu"))
model_first.add(Dropout(0.50))

model_first.add(Dense(3, activation = "softmax"))
     
    
model_first.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Second Model

In [ ]:
model_second= Sequential()
model_second.add(Conv2D(32, kernel_size = (5,5),padding = 'Same',strides=(2, 2),
                 activation ='relu', input_shape = (50,50,3) ))
model_second.add(Conv2D(64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model_second.add(MaxPooling2D(5,5,padding='same'))
model_second.add(Dropout(0.25))

model_second.add(Conv2D(128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model_second.add(Conv2D(256, kernel_size = (5,5),padding = 'Same', 
                  
                 activation ='relu'))

model_second.add(Dropout(0.50))

model_second.add(Flatten())
model_second.add(Dense(128, activation = "relu"))
model_second.add(Dropout(0.50))

model_second.add(Dense(3, activation = "sigmoid"))
     
    
model_second.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip=True)

test_datagen =  ImageDataGenerator(rescale = 1./255)

In [ ]:
train_set = train_datagen.flow_from_directory('dataset/train',
                                             target_size =(50,50),
                                             batch_size=32,
                                             class_mode='categorical')

test_set = train_datagen.flow_from_directory('dataset/test',
                                             target_size =(50,50),
                                             batch_size=32,
                                             class_mode='categorical')

In [ ]:
#steps_per_epoch = len(X_train)
#validation_steps = len(X_test)//batch_size # if you have test data

hist1 = model_first.fit_generator(
train_set,
steps_per_epoch=int(8000/3750),
epochs=15,
validation_data = test_set,
validation_steps =int(500/3750))
 

In [ ]:
score_first = model_first.evaluate(test_set, verbose=0)
print('model Test accuracy: {0:%}'.format(score_first[1]))
print('model Test LOSS: {0:%}'.format(score_first[0]))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hist1.history['accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('fist_model_accuracy.jpeg')

In [ ]:
plt.plot(hist1.history['loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('first_model_loss.jpeg')

Secon Model Training


In [ ]:
hist2_2 = model_second.fit_generator(
train_set,
steps_per_epoch=int(8000/3750),
epochs=15,
validation_data = test_set,
validation_steps =int(400/3750))

In [ ]:
score_second = model_second.evaluate(test_set, verbose=0)
print('model Test accuracy: {0:%}'.format(score_second[1]))
print('model Test LOSS: {0:%}'.format(score_second[0]))

Face Recognition

In [ ]:
train_set.class_indices

In [ ]:
##xml dosyasını kullanılabilir hale getiriliyor.
face_cascade= cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
classes = ['merve','nehir','sevim']
import numpy as np
from PIL import Image

In [ ]:
#kamera açılıyor
cap = cv2.VideoCapture(0)
while True:
    _,frame = cap.read()
    # görüntüde yüz bulmak için detectMultiScale fonksiyonunu kullanıyoruz.
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    
    # her yüz için tespit yapıyor
    for (x,y,w,h) in faces:
        #yüzün çevresine çerçeve yerleştiriyoruz.
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
        face = frame[y:y+h, x:x+w]
        
        if type(face) is np.ndarray:
            face = cv2.resize(face, (50,50))
            #girdi olarak dizi nesnesini alır ve dizi nesnesinden oluşturulan görüntü nesnesini döndürür.

            im = Image.fromarray(face, 'RGB')
            img_array= np.array(im)
            img_array = np.expand_dims(img_array, axis=0)
            pred = model_second.predict(img_array)
            label = classes[pred.argmax()]
            
    cv2.putText(frame, str(label), (x,y), cv2.FONT_HERSHEY_SIMPLEX, .7,(0,255,0),2)
    cv2.imshow('frame', frame)
    k = cv2.waitKey(100) & 0xff # esc tuşuna basarak çıkış yapılır.
    if k == 27:
        break


cap.release()
cv2.destroyAllWindows()   
    